In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

import pandas as pd 
import ast
import numpy as np
import json
from tqdm import tqdm
from IPython.display import clear_output
import os
from typing import Dict, List

from src.Retriever import ThresholdRetriever
from src.Scorer import SimilarityScorerConfig
from src.Reader import LLM_Model
from src.utils import ReaderMetrics, RetrieverMetrics, save_rag_trial_log, prepare_thresholdretriever_configs, prepare_reader_configs, load_benchmarks_df
from src.utils import evaluate_retriever, evaluate_reader

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# !!! TO CHANGE !!!
TRIAL = 3
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

# Retriever part
RETRIEVER_PARAMS = {
    "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
    "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
    "model_kwargs": {'device':'cuda'},
    "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
    "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 3}
}

# Reader part
READER_PARAMS = {
    'prompts': {
        "assistant": 'Отвечай на вопросы, используя информацию из текстов в списке ниже. Каждому тексту в начале в квадратных скобках поставлена в соответствие вещественная оценка его семантической близости к вопросу: в диапозоне от 0.0 (высокая близость) до 1.0 (низкая близость). Используй эту информацию. Выбирай тексты c достаточно высокими оценками близости для генерации ответа на их основе. Если на основании указанных оценок близости ты не уверена в релевантности данных текстов по отношению к заданному вопросу, то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.".',
        "system": "Ты вопросно-ответная система. Все ответы генерируй на русском языке. По вопросам отвечай кратко, чётко и конкретно. Не генерируй излишнюю информацию.",
    },
    'gen': {'max_new_tokens': 512, 'eos_token_id': 79097},
    'data_operate': {'batch_size': 1}
    }

ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}

BERTSCORE_MODEL_PATH = "ru_electra_medium"
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_READERCACHE = f'{SAVE_LOGDIR}/reader_cache.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

##### Configure Retriever-part

In [3]:
retrievers_config, benchmarks_path = prepare_thresholdretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
retriever_metrics = RetrieverMetrics()

In [4]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

##### Configure Reader-part

In [1]:
reader_config = prepare_reader_configs(READER_PARAMS)
READER = LLM_Model(reader_config)

NameError: name 'prepare_reader_configs' is not defined

In [6]:
sim_score_config = SimilarityScorerConfig()
reader_metrics = ReaderMetrics(BASE_DIR, BERTSCORE_MODEL_PATH, sim_score_config, READER)

In [8]:
RETRIEVERS = {name: ThresholdRetriever(config) for name, config in retrievers_config.items()}

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


##### Evaluating pipeline

In [12]:
retriever_scores, retriever_cache_ids, predicted_chunks, cache_relevant_flags = evaluate_retriever(
    benchmarks_df, RETRIEVERS, retriever_metrics, show_step=5, topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

mtssquad


100%|██████████| 500/500 [00:08<00:00, 62.45it/s, MRR=0.778, mAP=0.778, Recall=0.839, Precision=0.28, F1=0.419, NoRelContextScore=nan] 


In [21]:
contexts = {name: [reader_config.prompts.assistant + "\n\n" + "\n\n".join([f'{i+1}. [{round(doc[0], 3)}] {doc[1]}' for i, doc in enumerate(docs)]) 
                                                                      for docs in chunks] for name, chunks in predicted_chunks.items()}

In [31]:
reader_scores, reader_cache = evaluate_reader(benchmarks_df, READER, reader_metrics, contexts, show_step=1, cache_relevant_flags=cache_relevant_flags)

0it [00:00, ?it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


False
У меня нет ответа на ваш вопрос.
Да, по просьбе Мадонны заменили аранжировщика Каминса на более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса.


1it [00:04,  4.48s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=1]

[1]
False
"Everybody"
Песню Мадонны "Everybody" выпускают в качестве сингла.


2it [00:06,  3.32s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=0.5]

...

True
В 1910 году писатель Александр Грин был сослан в Пинегу, Архангельскую губернию.
После третьего ареста писатель Александр Грин был сослан в Пинегу Архангельской губернии.
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]


499it [28:55,  1.40s/it, BLEU2=0.28, BLEU1=0.341, ExactMatch=0.0191, METEOR=0.519, BertScore=nan, StubScore=0.438]

True
Нансен
О том, что айсберги могут переносить значительные количества минерального материала, дающего представление о геологическом строении неисследованных областей Гренландии, обнаружил Фритьоф Нансен.
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]


500it [28:56,  3.47s/it, BLEU2=0.279, BLEU1=0.34, ExactMatch=0.019, METEOR=0.518, BertScore=nan, StubScore=0.438] 

True
27 июня.
27 июня Викинг Нансена оказался затёрт сплошными ледовыми полями, начался незапланированный дрейф.
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]


In [32]:
save_rag_trial_log(SAVE_LOGDIR, reader_scores, retriever_scores, 
                   SAVE_HYPERPARAMS, SAVE_READERCACHE, SAVE_RETRIEVERCACHE,
                   reader_cache, retriever_cache_ids, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE,
                   READER_PARAMS, RETRIEVER_PARAMS, ADDITIONAL_PARAMS)

In [33]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.10/asynci